In [ ]:
import torch
import numpy as np
from graphqec.qecc import get_code
from graphqec.decoder.nn.train_utils import build_neural_decoder

example_config = {
    "code": {
        "code_type": "TriangleColorCode",
        "distance": 9
    },
    "decoder": {
        "name": "GraphLinearAttnDecoderV2A",
        "encoder_dim": 192,
        "decoder_dim": 256,
        "readout_dim": 192,
        "num_encoder_layers": 5,
        "num_decoder_layers": 4,
        "num_readout_layers": 16,
        "num_heads": 8,
        "scatter_activation": "tanh",
        "scatter_fn": "mul",
        "ffn_dim_multiplier": 3.0,
        "multiple_of": 32,
        "norm_eps": 1e-05,
        "chkpt": "/home/hugengyuan/projects/graphqec/tmp/results/TCd9/pretrain_latest" # subsitute with your own checkpoint path
    },
}

test_code = get_code(
    name=example_config['code']["code_type"], 
    **example_config['code']
)

graphqec_decoder = build_neural_decoder(
    test_code.get_tanner_graph().to(torch.device("cuda")),
    example_config['decoder']
    ).to(device="cuda", dtype=torch.bfloat16) # run with bf16 is much faster and almost the same accurate as fp32

/home/hugengyuan/miniconda3/envs/qecnew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


matched keys: 631, not matched keys: 0
not matched keys: []


In [2]:
# num_cycle begins with 0, so 8 cycles means 9 rounds(one init round + 8 cycle rounds)
sampler = test_code.get_dem(num_cycle=8,physical_error_rate=0.005).compile_sampler() 
syndromes, obs_flips, _ = sampler.sample(10000)
preds = graphqec_decoder.decode(syndromes, batch_size=100)

print(np.mean(preds==obs_flips))

0.951
